In [17]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [18]:
uber_csv = pd.read_csv("Resources/uber.csv")

uber_df = pd.DataFrame(uber_csv)

uber_df.head()



Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1  
1         -73.994710         40.750325                1  
2         -73.962565         40.772647                1  
3         -73.965316         40.803349                3  
4         -73.973082         40.761247                5

In [19]:
uber_df.rename(columns = {'Unnamed: 0':'ID', 'key':'N/A', 'fare_amount':'Charge', "pickup_datetime":"Pickup Time",  "pickup_longitude": "Pickup Lon", "pickup_latitude":"Pickup Lat",  "dropoff_longitude": "Dropoff Lon", "dropoff_latitude": "Dropoff Lat", "passenger_count": "Passenger Count"}, inplace = True)

uber_df.head()



ID                            N/A  Charge              Pickup Time  \
0  24238194    2015-05-07 19:52:06.0000003     7.5  2015-05-07 19:52:06 UTC   
1  27835199    2009-07-17 20:04:56.0000002     7.7  2009-07-17 20:04:56 UTC   
2  44984355   2009-08-24 21:45:00.00000061    12.9  2009-08-24 21:45:00 UTC   
3  25894730    2009-06-26 08:22:21.0000001     5.3  2009-06-26 08:22:21 UTC   
4  17610152  2014-08-28 17:47:00.000000188    16.0  2014-08-28 17:47:00 UTC   

   Pickup Lon  Pickup Lat  Dropoff Lon  Dropoff Lat  Passenger Count  
0  -73.999817   40.738354   -73.999512    40.723217                1  
1  -73.994355   40.728225   -73.994710    40.750325                1  
2  -74.005043   40.740770   -73.962565    40.772647                1  
3  -73.976124   40.790844   -73.965316    40.803349                3  
4  -73.925023   40.744085   -73.973082    40.761247                5

In [20]:
uber_df3 = uber_df.sample(n=5)
uber_df3

ID                            N/A  Charge  \
195063  20132191  2013-05-19 00:48:00.000000154    11.0   
53687   51954756    2014-06-19 18:45:46.0000003     3.5   
114992    149800    2010-04-27 21:40:54.0000003    10.9   
31255    5758919    2009-06-23 08:41:05.0000003    13.7   
159420  22846831   2011-01-31 11:08:00.00000089    39.7   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
195063  2013-05-19 00:48:00 UTC  -73.975230   40.751822   -73.978000   
53687   2014-06-19 18:45:46 UTC  -73.974295   40.778620   -73.977006   
114992  2010-04-27 21:40:54 UTC  -73.989819   40.729577   -73.953298   
31255   2009-06-23 08:41:05 UTC  -73.987299   40.768614   -74.007379   
159420  2011-01-31 11:08:00 UTC  -73.790077   40.643858   -73.939735   

        Dropoff Lat  Passenger Count  
195063    40.725327                1  
53687     40.774740                1  
114992    40.772547                1  
31255     40.727838                1  
159420    40.751470                1

In [27]:
uber_df3['Date Formatted'] = (pd.to_datetime(uber_df3['Pickup Time'], yearfirst=True, utc=True)) 
x = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s') 
uber_df3


ID                            N/A  Charge  \
195063  20132191  2013-05-19 00:48:00.000000154    11.0   
53687   51954756    2014-06-19 18:45:46.0000003     3.5   
114992    149800    2010-04-27 21:40:54.0000003    10.9   
31255    5758919    2009-06-23 08:41:05.0000003    13.7   
159420  22846831   2011-01-31 11:08:00.00000089    39.7   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
195063  2013-05-19 00:48:00 UTC  -73.975230   40.751822   -73.978000   
53687   2014-06-19 18:45:46 UTC  -73.974295   40.778620   -73.977006   
114992  2010-04-27 21:40:54 UTC  -73.989819   40.729577   -73.953298   
31255   2009-06-23 08:41:05 UTC  -73.987299   40.768614   -74.007379   
159420  2011-01-31 11:08:00 UTC  -73.790077   40.643858   -73.939735   

        Dropoff Lat  Passenger Count            Date Formatted  
195063    40.725327                1 2013-05-19 00:48:00+00:00  
53687     40.774740                1 2014-06-19 18:45:46+00:00  
114992    40.772547                1 2010-04-27 21:40:54+00:00  
31255     40.727838                1 2009-06-23 08:41:05+00:00  
159420    40.751470                1 2011-01-31 11:08:00+00:00

In [30]:
uber_df3['Date Formatted Final'] = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s')
uber_df3



ID                            N/A  Charge  \
195063  20132191  2013-05-19 00:48:00.000000154    11.0   
53687   51954756    2014-06-19 18:45:46.0000003     3.5   
114992    149800    2010-04-27 21:40:54.0000003    10.9   
31255    5758919    2009-06-23 08:41:05.0000003    13.7   
159420  22846831   2011-01-31 11:08:00.00000089    39.7   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
195063  2013-05-19 00:48:00 UTC  -73.975230   40.751822   -73.978000   
53687   2014-06-19 18:45:46 UTC  -73.974295   40.778620   -73.977006   
114992  2010-04-27 21:40:54 UTC  -73.989819   40.729577   -73.953298   
31255   2009-06-23 08:41:05 UTC  -73.987299   40.768614   -74.007379   
159420  2011-01-31 11:08:00 UTC  -73.790077   40.643858   -73.939735   

        Dropoff Lat  Passenger Count            Date Formatted  \
195063    40.725327                1 2013-05-19 00:48:00+00:00   
53687     40.774740                1 2014-06-19 18:45:46+00:00   
114992    40.772547                1 2010-04-27 21:40:54+00:00   
31255     40.727838                1 2009-06-23 08:41:05+00:00   
159420    40.751470                1 2011-01-31 11:08:00+00:00   

        Date Formatted Final  
195063            1368924480  
53687             1403203546  
114992            1272404454  
31255             1245746465  
159420            1296472080

In [32]:
clean_df = uber_df3[["ID", "Charge", "Pickup Lon", "Pickup Lat", "Dropoff Lon", "Dropoff Lat", "Passenger Count", "Date Formatted Final" ]]
clean_df


ID  Charge  Pickup Lon  Pickup Lat  Dropoff Lon  Dropoff Lat  \
195063  20132191    11.0  -73.975230   40.751822   -73.978000    40.725327   
53687   51954756     3.5  -73.974295   40.778620   -73.977006    40.774740   
114992    149800    10.9  -73.989819   40.729577   -73.953298    40.772547   
31255    5758919    13.7  -73.987299   40.768614   -74.007379    40.727838   
159420  22846831    39.7  -73.790077   40.643858   -73.939735    40.751470   

        Passenger Count  Date Formatted Final  
195063                1            1368924480  
53687                 1            1403203546  
114992                1            1272404454  
31255                 1            1245746465  
159420                1            1296472080